In [53]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')

In [54]:
dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])

dat['b_category'] = dat['assessmentItemID'].str[2]
dat['test_category'] = dat['assessmentItemID'].str[4:7]

_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [55]:
user_final_time = _train.groupby('userID')['Timestamp'].max()
_train['train_valid'] = _train.apply(lambda x : -1 if x.Timestamp == user_final_time[x.userID] else x['answerCode'], axis = 1)

/tmp/ipykernel_23775/3440697663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _train['train_valid'] = _train.apply(lambda x : -1 if x.Timestamp == user_final_time[x.userID] else x['answerCode'], axis = 1)


In [56]:
_valid = _train[_train['train_valid'] < 0]
_train = _train[_train['train_valid'] >= 0]

In [57]:
_test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,b_category,test_category
1035,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289,5,133
1706,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080,7,146
3023,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660,7,111
4283,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611,9,064
4670,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422,6,135
...,...,...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615,4,122
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636,3,111
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402,5,193
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402,5,193


In [59]:
_train_pred = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True)

_valid_pred = _valid['answerCode']
_valid.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True)

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True)

/tmp/ipykernel_23775/1621781643.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True)


In [63]:
_train['userID'] = _train['userID'].astype('str')
_train['KnowledgeTag'] = _train['KnowledgeTag'].astype('str')

_valid['userID'] = _valid['userID'].astype('str')
_valid['KnowledgeTag'] = _valid['KnowledgeTag'].astype('str')

_test['userID'] = _test['userID'].astype('str')
_test['KnowledgeTag'] = _test['KnowledgeTag'].astype('str')

/tmp/ipykernel_23775/362564649.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _test['userID'] = _test['userID'].astype('str')
/tmp/ipykernel_23775/362564649.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _test['KnowledgeTag'] = _test['KnowledgeTag'].astype('str')


In [71]:
#!pip install catboost
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(learning_rate=0.05, iterations=1000, task_type="GPU")
cat_model.fit(_train, _train_pred, early_stopping_rounds=100, cat_features=list(_train.columns) ,verbose=500)
pred = cat_model.predict_proba(_valid)[:,1]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 24.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 70.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
0:	learn: 0.6766703	total: 62.1ms	remaining: 1m 2s
500:	learn: 0.4688769	total: 35.3s	remaining: 35.2s
999:	learn: 0.4671602	total: 1m 8s	remaining: 0us


AttributeError: 'CatBoostClassifier' object has no attribute 'pred'

In [105]:
_test_pred = cat_model.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred

/tmp/ipykernel_23775/1090926449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _test['prediction'] = _test_pred


In [108]:
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.to_csv('output/cat_submission.csv', index = False)